In [83]:
# import necessary libraries
from datetime import date, datetime, timedelta
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

DB_NAME = 'timperio'
USERNAME = 'root'
PASSWORD = 'password'
HOST = 'localhost'

# connect to the database
try:
    connection = mysql.connector.connect(host=HOST, user=USERNAME, password=PASSWORD, database=DB_NAME)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your username/password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
        
cursor = connection.cursor()

In [84]:
# Load the data from Excel and convert the date format
df_sales = pd.read_csv('CSV_Sales_Data_updated_Sep 2023.csv')
# df_sales['Sale Date'] = pd.to_datetime(df_sales['Sale Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Convert 'Sale Date' to the proper format if needed
df_sales['Sale Date'] = pd.to_datetime(df_sales['Sale Date'], errors='coerce').dt.strftime('%Y-%m-%d')
df_sales = df_sales.replace({float("NaN"): None})

/var/folders/75/0wdzm0qd6qqgtzrtbxszyss80000gn/T/ipykernel_88544/757648862.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_sales['Sale Date'] = pd.to_datetime(df_sales['Sale Date'], errors='coerce').dt.strftime('%Y-%m-%d')


In [85]:
# Insert the data into the database
# Have to insert data in this order: Customer, Product, Sale

# Insert data into Customer table
try:
    for index, row in df_sales.iterrows():
        sql = """
        INSERT INTO customers (customer_id, zipcode) VALUES (%s, COALESCE(%s, 0))
        ON DUPLICATE KEY UPDATE 
            `zipcode` = CASE
                WHEN VALUES(zipcode) IS NOT NULL THEN VALUES(zipcode)
                ELSE zipcode
            END;
        """
        
        values = (
            row['Customer ID'],
            row['ZipCode']
        )
        cursor.execute(sql, values)
        connection.commit()
except mysql.connector.Error as err:
    print(err)

In [86]:
def insert_into_sequence(table_name, next_id):
    try:
        cursor.execute("INSERT IGNORE INTO sequence (table_name, next_id) VALUES (%s, %s)", (table_name, next_id))
        connection.commit()  # Commit transaction
    except mysql.connector.Error as err:
        print(f"Error: {err}")

# Call the function
insert_into_sequence('product_id', 0)

In [87]:
# Insert data into Product table
try:
    for index, row in df_sales.iterrows():
        # Check if there is a product with the same name and variant
        cursor.execute("SELECT product_id FROM products WHERE product_name = %s AND product_variant = %s", (row['Product'], row['Variant']))
        product = cursor.fetchone()
        
        if product == None: # if there is no product with the same name and variant
            # Lock the sequence row for safe access
            cursor.execute("SELECT next_id FROM sequence WHERE table_name = 'product_id' FOR UPDATE")
            current_id = cursor.fetchone()[0]
            
            # Increment the ID
            new_id = current_id + 1
            cursor.execute("UPDATE sequence SET next_id = %s WHERE table_name = 'product_id'", (new_id,))
            
            # Insert the new product
            sql = """
            INSERT INTO products (product_id, product_name, product_variant, product_price) VALUES (%s, %s, %s, %s)
            """
            # ON DUPLICATE KEY UPDATE `product_price` = VALUES(product_price);
            # """
        
            values = (
                new_id,
                row['Product'],
                row['Variant'],
                row['Price']
            )
            cursor.execute(sql, values)
            connection.commit()
except mysql.connector.Error as err:
    connection.rollback()
    print(err)

In [88]:
# Insert data into Sales Table
insert_into_sequence('sale_id', 0)

try:
    for index, row in df_sales.iterrows():
            # Determine the product
            cursor.execute("SELECT product_id FROM products WHERE product_name = %s AND product_variant = %s", (row['Product'], row['Variant']))
            product_id = cursor.fetchone()[0]
            
            # fix the date format
        
            # Insert the new sale
            sql = """
            INSERT IGNORE INTO sales (sale_date, sale_type, digital, shipping_method, product_id, quantity, original_price, customer_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
        
            values = (
                row['Sale Date'],
                row['Sale Type'],
                row['Digital'],
                row['Shipping Method'],
                product_id,
                row['Quantity'],
                row['Product Price'],
                row['Customer ID']
            )
            cursor.execute(sql, values)
            connection.commit()
except mysql.connector.Error as err:
    connection.rollback()
    print(err)

In [89]:
# close cursor and connection
cursor.close()
connection.close()